# Homework 2 for CS 247 : Advanced Data Mining Learning

### Due: 11:59 pm 04/14


__Name__: Chiao Lu

__UID__: 204848946

## Problem 1:  Logistic Regression

1. Write down the matrix form operation for gradient vector and Hessian matrix for logistic regression, according to Slide 35 and Slide 36;


#### Write Your answer here:
gradient:
<br>
Let $$X\in \mathbb{R}^{N\times (p+1)}$$, $$Y\in \mathbb{R}^{N}$$, and $$p(\beta) \in \mathbb{R}^{N}$$ where $$p(\beta)=\sigma (\beta^{T} X)$$
<br>
$$\frac{\partial \log L(\beta)}{\partial \beta}=X^{T} \times (Y-p(\beta))$$
<br>
<br>
Hessian:
<br>
Let $$\mathrm{diag} (A)$$ be the same dimension as $$A$$ such that $$(\mathrm{diag} (A))_{ij}=\left\{\begin{matrix}A_{ij}&i=j\\0&i\neq j\\\end{matrix}\right.$$ 
<br>
Then 
$$\frac{\partial^{2} \log L(\beta)}{\partial \beta^{2}}=-X^{T} \times \mathrm{diag(p(\beta))} \times \mathrm{diag(1-p(\beta))} \times X$$

2. When implementing ML algorithms, it is important to paralize numerical computation and avoid for-loop computation. For optimizing Logistic Regression, we've alreay implemented the ***for-loop*** version of ***Gradient Ascent*** algorithm. Please implement ***matrix*** version of ***Gradient Ascent***, implement ***My_Logistic_Regression*** for training and prediction. Finally, Compare the accuracy and running time of the sklearn Logistic Regression with these two optimization.

3. Implement ***Newton's Method*** (matrix version), compare its accuracy, running time with ***Gradient Ascent***

In [1]:
from sklearn.metrics import classification_report 
import numpy as np
import time
   
    
def sigmoid(z):
    return 1 / (1 + np.exp(-z)) 
# ==================
#  Three different Optimizers
# ==================


def forloop_GA_optimizer(W, X, y):
    """
    Gradient Ascent (GA) optimizer implemented with for loop.

    Args:
        W - Parameters
        X - Features of training batch/instance
        y - Label(s) of training batch/instance
    Return:
        W_new - Updated W
    """
    learning_rate = 1e-2
    n_data, n_features = X.shape[0], X.shape[1] #total number of attributes
    gradient = np.zeros(n_features)
    for j in range(n_features):
        for i in range(n_data):
            gradient[j] += X[i][j] * (y[i] - sigmoid(np.dot(np.transpose(W), X[i])))
    W += learning_rate * gradient



def matrix_GD_optimizer(W, X, y):
    """
    Gradient Ascent (GA) optimizer implemented with matrix operations.
    """
    
    # ========= TODO start ==========
    learning_rate = 1e-2
    gradient = np.transpose(X) @ (y - sigmoid(W@X))
    W += learning_rate*gradient
    # ========= TODO end ============



def matrix_Newton_optimizer(W, X, y):
    """
    Newton's method optimizer implemented with matrix operations.
    """
    
    # ========= TODO start ==========
    hessian = -np.transpose(X)@np.diag(sigmoid(W@X))@np.diag(1 - sigmoid(W@X))@X
    hessian_inv = np.linalg.inv(hessian)
    gradient = np.transpose(X) @ (y - sigmoid(W@X))
    W -= hessian_inv@gradient
    # ========= TODO end ============
    
class My_Logistic_Regression():
    """
    Parameters
    ----------
        n_features : int, feature dimension
        optimizer  : function, one optimizer that takes input the model weights
                     and training data to perform one iteration of optimization.
    """
    def __init__(self, n_features, optimizer=matrix_GD_optimizer):
        self.W = np.random.rand(n_features)
        self.optimizer = optimizer
    def fit(self, X, y):
        n_epoch = 10000
        for epoch in range(n_epoch):
            self.optimizer(self.W, X, y)
    def predict(self, X):
        pass
    
    def predict_proba(self, X):
        pass

SyntaxError: invalid syntax (<ipython-input-1-a3f43d1dcd12>, line 41)

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

X, y = load_breast_cancer(return_X_y=True)
n_data, n_features = X.shape[0], X.shape[1]
ids  = np.arange(n_data)
np.random.shuffle(ids)
train_ids, test_ids = ids[: n_data // 2], ids[n_data // 2: ]

train_X, train_y = X[train_ids], y[train_ids]
test_X,  test_y  = X[test_ids],  y[test_ids]
clf = LogisticRegression()
start_time = time.time()
clf.fit(train_X, train_y)
end_time = time.time()
print('Training time: %d s' % (end_time - start_time))
pred_y = clf.predict(test_X)
print(classification_report(pred_y, test_y))


for optimizer in [forloop_GA_optimizer, matrix_GD_optimizer]:
    my_clf = My_Logistic_Regression(n_features, optimizer)
    start_time = time.time()
    my_clf.fit(train_X, train_y)
    end_time = time.time()
    print('Training time: %d s' % (end_time - start_time))
    pred_y = my_clf.predict(test_X)
    print(classification_report(pred_y, test_y))

## Problem 2 Poisson Regressions

Poisson regression is another example of generalized linear model, which assumes the label $y$ has a Poisson distribution.

1. Prove Poisson distribution belongs to exponential family;

#### Write Your answer here:

$$P(Y=y)=\frac{\lambda^{y}\exp(-\lambda)}{y!}$$
$$=\frac{\exp(\log{\lambda^{y}})\exp(-\lambda)}{y!}=\frac{\exp(y \log{\lambda})\exp(-\lambda)}{y!}=\frac{\exp(y \log(\lambda) - \lambda)}{y!}$$
<br>
Now, let $$b(y)=\frac{1}{y!}$$, $$\eta=\log(\lambda)$$, $$T(y)=y$$, and $$a(\eta)=\exp(\eta)$$.
<br>
With the above substitution, we get $$b(y)\exp( \eta^{T} T(y)-a(\eta))$$. This tells us that Poission belongs to exponential family.

2. Follow the recipe of GLM, write down Poisson regression model. Then write down how to train the model given a dataset $D = \{x_i, y_i\}^n_{i=1}$, following MLE (Maximum Likelihood Estimation) paradigm.

#### Write Your answer here:
OK.
<br>
Substitue $$\eta=\log(\lambda)=\beta^{T} x$$ in the above $$P(Y=y)$$ and we get $$P(Y=y)=\frac{\exp(y \beta^{T}x - \exp(\beta^{T} x))}{y!}$$
<br>
The joint probablity of observing the training data is then<br>
$$P(y_{1}, y_{2},...y_{n}|x_{1}, x{2},...,X_{n}, \beta)=\prod_{i=1}^{n}\frac{\exp(y_{i} \beta^{T}x_{i} - \exp(\beta^{T} x_{i}))}{y_{i}!}$$
<br>
The log-likelihood is then $$\mathrm{LL}=\sum_{i=1}^{n}y_{i} \beta^{T}x_{i} - \exp(\beta^{T} x_{i})-\log(y_{i}!)$$
<br>
We wish to find $${\arg \max}_{\beta}\mathrm{LL}(\beta)$$.
To do MLE we take gradient of $$\mathrm{LL}$$ with respect to $$\beta$$.
<br>
We have $$\frac{\partial \mathrm{LL}(\beta)}{\partial \beta} = \sum_{i=1}^{n}y_{i} x_{i} - x_{i} \exp(\beta^{T} x_{i}) = \sum_{i=1}^{n} x_{i} (y_{i}-\exp(\beta^{T} x_{i}))$$
<br>
With the gradient, we can now perform gradient descent!